# Small Test Set #
## Goal ##
The goal of this test set is to perform proof of concept testing on a small number of proteins with a wide range of sizes and available homologs, orthologs, and paralogs. By doing so it should be possible to test the best parameterization for this tool as well as identifying the strengths and weaknesses of the tool using various measurments as end points.
## Warning ##
Before attempting to use this notebook make sure that your .env file has been properly setup to reflect the correct locations of command line tools and the location of files and directories needed for execution.
### Initial Import###
This first cell performs the necessary imports required to begin this notebook.

In [1]:
from dotenv import find_dotenv, load_dotenv
try:
    dotenv_path = find_dotenv(raise_error_if_not_found=True)
except IOError:
    dotenv_path = find_dotenv(raise_error_if_not_found=True, usecwd=True)
load_dotenv(dotenv_path)
import os
import sys
sys.path.append(os.path.join(os.environ.get('PROJECT_PATH'), 'src'))
sys.path.append(os.path.join(os.environ.get('PROJECT_PATH'), 'src', 'SupportingClasses'))
input_dir = os.environ.get('INPUT_PATH')

## Data Set Construction ##
The first task required to test the data set is to download the required data and construct any necessary input files for all down stream analyses.
In this case that means:
* Downloading PDB files for the proteins in our small test set.
* Extracting a query sequence from each PDB file.
* Searching for paralogs, homologs, and orthologs in a custom BLAST database built by filtering the Uniref90 database.
* Filtering the hits from the BLAST search to meet minimum and maximum length requirements, as well as minimum and maximum identity requirements.
* Building alignments using CLUSTALW in both the fasta and msf formats since some of the tools which will be used for comparison need different formats.
* Filtering the alignment for maximum identity similarity between seqeunces.
* Re-aligning the filtered sequences using CLUSTALW.
This is all handeled by the DataSetGenerator class found in the src/SupportingClasses folder

In [2]:
from time import time
from DataSetGenerator import DataSetGenerator
protein_list_dir = os.path.join(input_dir, 'ProteinLists')
if not os.path.isdir(protein_list_dir):
    os.makedirs(protein_list_dir)
small_list_fn = os.path.join(protein_list_dir, 'SmallDataSet.txt')
if not os.path.isfile(small_list_fn):
    proteins_of_interest = ['2ysdA', '1c17A', '3tnuA', '7hvpA', '135lA', '206lA', '2werA', '1bolA', '3q05A', '1axbA',
                            '2rh1A', '1hckA', '3b6vA', '2z0eA', '1jwlA', '1a26A', '1c0kA', '4lliA', '4ycuA', '2iopA',
                            '2zxeA', '2b59B', '1h1vG']
    with open(small_list_fn, 'w') as small_list_handle:
        for p_id in proteins_of_interest:
            small_list_handle.write('{}\n'.format(p_id))
generator = DataSetGenerator(input_dir)
start = time()
summary = generator.build_pdb_alignment_dataset(protein_list_fn=os.path.basename(small_list_fn), processes=10,
                                                database='customuniref90.fasta', max_target_seqs=2500, remote=False,
                                                sources=['PDB'], verbose=False)
summary['Chain'] = summary['Protein_ID'].apply(lambda x: generator.protein_data[x]['Chain'])
summary['Accession'] = summary['Protein_ID'].apply(lambda x: generator.protein_data[x]['Accession'])
summary['Length'] = summary['Protein_ID'].apply(lambda x: generator.protein_data[x]['Length'])
summary['Total_Size'] = summary.apply(lambda x: float(x['Length']) * float(x['Filtered_Alignment']), axis=1)
summary.sort_values(by=['Filtered_Alignment', 'Length'], axis=0, inplace=True)
summary_columns = ['Protein_ID', 'Chain', 'Accession', 'BLAST_Hits', 'Filtered_BLAST',
                   'Filtered_Alignment', 'Length', 'Total_Size']
print(summary[summary_columns])
end = time()
print('It took {} min to generate the data set.'.format((end - start) / 60.0))
summary.to_csv(os.path.join(input_dir, 'small_data_set_summary.tsv'), sep='\t', index=False, header=True,
               columns=summary_columns)

Importing protein list
Unique Sequences Found: 23!
BLASTing query sequences
Filtering BLAST hits, aligning, filtering by identity, and re-aligning
   Protein_ID Chain Accession  BLAST_Hits  Filtered_BLAST  Filtered_Alignment  \
20       2rh1     A      None        2500              35                  35   
7        1c0k     A      None        2500              54                  54   
12       7hvp     A      None         128              60                  59   
10       2b59     B      None         327              79                  73   
3        206l     A      None        1193             102                  93   
11       1bol     A      None        2500             158                 154   
17       1jwl     A      None        2500             234                 231   
0        3q05     A      None         939             468                 316   
21       2z0e     A      None        2106             393                 351   
6        4lli     A      None        2500  

Create a location to store the output of this method comparison.

In [3]:
output_dir = os.environ.get('OUTPUT_PATH')
small_set_out_dir = os.path.join(output_dir, 'SmallTestSet')
if not os.path.isdir(small_set_out_dir):
    os.makedirs(small_set_out_dir)

## Setting Up Scoring For Each Method
To reduce memory load during prediction and evaluation, the scoring objects needed to compute the metrics used to compare methods will be created ahead of time so they are available to each method when it computes its predictions for a given protein. This will ensure that results do not need to be kept in memory while waiting for all other results to be computed, only the metrics measured for each method will be recorded.

# Generating Values For Comparision#
To determine the effectiveness of the new method and implementation the covariation of the same proteins will be computed using the previous Evolutionary Trace covariation method (ET-MIp) and other methods in the field.

## ET-MIp##
Scoring the the covariation of the proteins using the previous Evolutionary Trace covariation method (ET-MIp).

## ET-MIp (Continued)
The previous implementation is not able to run for alignments of the size used here. Instead we use the new implementation with the same parameterization used by the previous implementation (Distance Model - blosum62 similarity, Tree - ET UPGMA variant, Scoring Metric - filtered average product corrected mutual information, Ranks - all).

## cET-MIp
This segment the ET-MIp method, when constrained to an arbitrary set of nodes (1, 2, 3, 5, 7, 10, 25) at the top of the phylogenetic tree.

## ET-MIp with Group Maximiziation ##
This cell generates data for and tests the effect of maximizing the group score when moving from a parent node to child nodes.

## DCA##
Scoring the the covariation of the proteins using a DCA julia implementation.

## EVCouplings##
Scoring the the covariation of the proteins using the EVCouplings method standard protocol.

Scoring the covariation of the proteins using the EVCouplings method mean field protocol.

In [ ]:
# Import the necessary packages to calculate and evaluate covariation predictions
import numpy as np
import pandas as pd
from SeqAlignment import SeqAlignment
from PDBReference import PDBReference
from utils import compute_rank_and_coverage
from ContactScorer import ContactScorer, plot_z_scores
from DCAWrapper import DCAWrapper
from EvolutionaryTrace import EvolutionaryTrace
from EVCouplingsWrapper import EVCouplingsWrapper
# Define the order of proteins, method, and separations for experiments and plotting
protein_order = list(summary['Protein_ID'])
method_order = ['DCA', 'EVC Standard', 'EVC Mean Field', 'ET-MIp', 'cET-MIp', 'ET-MMEA']
sequence_separation_order = ['Any', 'Neighbors', 'Short', 'Medium', 'Long']
# Define the predictor, settings, and rank type for each method to be tested
general_et_settings = {'polymer_type': 'Protein', 'et_distance': True, 'distance_model': 'blosum62', 'tree_building_method': 'et',
                 'tree_building_options': {}, 'position_type': 'pair', 'gap_correction': None,
                 'output_files': {'original_aln', 'non_gap_aln', 'tree', 'scores'}, 'processors': 10, 'low_memory': True}
method_dict = {'DCA': {'Predictor': DCAWrapper, 'Settings': {}, 'RankType':'max', 'Scoring Params': {}},
               'EVC Standard': {'Predictor': EVCouplingsWrapper, 'Settings': {'protocol': 'standard'}, 'RankType':'max', 'Scoring Params': {'cores': 10}},
               'EVC Mean Field': {'Predictor': EVCouplingsWrapper, 'Settings': {'protocol': 'mean_field'}, 'Scoring Params': {'cores': 10}},
               'ET-MIp': {'Predictor': EvolutionaryTrace, 'RankType':'max', 'Scoring Params': {},
                          'Settings': {'ranks': None, 'scoring_metric': 'filtered_average_product_corrected_mutual_information'}},
               'cET-MIp': {'Predictor': EvolutionaryTrace, 'RankType':'max', 'Scoring Params': {},
                           'Settings': {'ranks': [1, 2, 3, 5, 7, 10, 25], 'scoring_metric': 'filtered_average_product_corrected_mutual_information'}},
               'ET-MMEA': {'Predictor': EvolutionaryTrace, 'RankType':'max', 'Scoring Params': {},
                           'Settings': {'ranks': None, 'scoring_metric': 'match_mismatch_entropy_angle'}}}
method_dict['ET-MIp']['Settings'].update(general_et_settings)
method_dict['cET-MIp']['Settings'].update(general_et_settings)
method_dict['ET-MMEA']['Settings'].update(general_et_settings)
# Define the overall dataframe for evaluations, including its path and columns
small_comparison_df = None
small_comparison_fn = os.path.join(small_set_out_dir, 'Small_Comparision_Data.csv')
output_columns = ['Protein', 'Protein Length', 'Alignment Size', 'Method', 'Distance', 'Total Time', 
                  'Sequence_Separation', 'AUROC', 'AUPRC',
                  'Top K Predictions', 'Precision', 'Recall', 'F1 Score',
                  'Biased Z-Score at 10%', 'Biased Z-Score at 30%', 'Max Biased Z-Score', 'AUC Biased Z-Score',
                  'Unbiased Z-Score at 10%', 'Biased Z-Score at 30%', 'Max Unbiased Z-Score', 'AUC Unbiased Z-Score']
if os.path.isfile(small_comparison_fn):
    # If the full evaluation has already been completed load it
    small_comparison_df = pd.read_csv(small_comparison_fn, sep='\t', header=0, index_col=False)
else:
    # If not perform experiments for each protein
    for p_id in summary['Protein_ID']:
        # Define the protein result directory and dataframe path
        protein_dir = os.path.join(small_set_out_dir, p_id)
        if not os.path.isdir(protein_dir):
            os.makedirs(protein_dir)
        protein_df_fn =  os.path.join(protein_dir, '{}_Protein_Data.csv'.format(p_id))
        if os.path.isfile(protein_df_fn):
            # If this protein has already been evalauted load the evaluation
            protein_df = pd.read_csv(protein_df_fn, sep='\t', header=0, index_col=False)
        else:
            # Initialize the variable for the dataframe
            protein_df = None
            # Import alignment and remove gaps
            full_aln = SeqAlignment(file_name=generator.protein_data[p_id]['Final_FA_Aln'], query_id=p_id)
            full_aln.import_alignment()
            non_gap_aln = full_aln.remove_gaps()
            # Import structure
            pdb_structure = PDBReference(pdb_file=generator.protein_data[p_id]['PDB'])
            pdb_structure.import_pdb(structure_id=p_id)
            # Initialize Beta Carbon distance scorer
            contact_scorer_cb = ContactScorer(query=p_id, seq_alignment=non_gap_aln,
                                              pdb_reference=pdb_structure, cutoff=8.0)
            contact_scorer_cb.best_chain = generator.protein_data[p_id]['Chain']
            contact_scorer_cb.fit()
            contact_scorer_cb.measure_distance(method='CB')
            # Initialize distance scorer minimizing distance between any atoms
            contact_scorer_any = ContactScorer(query=p_id, seq_alignment=non_gap_aln,
                                               pdb_reference=pdb_structure, cutoff=8.0)
            contact_scorer_any.best_chain = generator.protein_data[p_id]['Chain']
            contact_scorer_any.fit()
            contact_scorer_any.measure_distance(method='Any')
            # Initialize z-scoring subproblems
            biased_w2_ave = None
            unbiased_w2_ave = None
            # Identify settings for this protein
            protein_settings = {'query': p_id, 'aln_file': generator.protein_data[p_id]['Final_FA_Aln']}
            # Then perform experiments for each method
            for method in method_order:
                if p_id == '2zxe' and method == 'EVC Mean Field':
                    continue
                # Set the result directory for this method
                method_dir = os.path.join(protein_dir, method)
                if not os.path.isdir(method_dir):
                    os.makedirs(method_dir)
                print('Attempting to calculate {} covariance for: {}'.format(method, p_id))
                # Define the path for the evaluation data for this experiment
                curr_fn = os.path.join(method_dir, '{}_{}_Method_Data.csv'.format(p_id, method))
                if os.path.isfile(curr_fn):
                    # If it has already been performed, load the data
                    curr_df = pd.read_csv(curr_fn, sep='\t', header=0, index_col=False)
                else:
                    # Otherwise perform the experiment, beginning with compiling the compelte settings for this predictor
                    curr_settings = {'out_dir': method_dir}
                    curr_settings.update(protein_settings)
                    curr_settings.update(method_dict[method]['Settings'])
                    predictor = method_dict[method]['Predictor'](**curr_settings)
                    total_time = predictor.calculate_scores(method_dict[method]['Scoring Params'])
                    print('Successfully computed {} covariance for: {}'.format(method, p_id))
                    # Compute statistics for the final scores of the ET-MIp model
                    curr_df, _, _ = contact_scorer_cb.evaluate_predictor(
                        predictor=predictor, verbosity=2, out_dir=method_dir, dist='CB', biased_w2_ave=None,
                        unbiased_w2_ave=None, processes=10, threshold=0.5, file_prefix='{}_Scores_'.format(method), plots=True)
                    # Score Prediction Clustering
                    contact_scorer_any.map_predictions_to_pdb(ranks=predictor.rankings, predictions=predictor.scores, coverages=predictor.coverages,
                                                              threshold=0.5)
                    z_score_fn = os.path.join(method_dir, '{}_Scores_Dist-Any_{}_ZScores.tsv'.format(method, p_id))
                    z_score_plot_fn = os.path.join(method_dir, '{}_Scores_Dist-Any_{}_ZScores.png'.format(method, p_id))
                    z_score_biased, biased_w2_ave, biased_scw_z_auc = contact_scorer_any.score_clustering_of_contact_predictions(
                        bias=True, file_path=z_score_fn.format('Biased'), w2_ave_sub=biased_w2_ave, processes=10)
                    biased_z_score_array = np.array(pd.to_numeric(z_score_biased['Z-Score'], errors='coerce'))
                    curr_df['Max Biased Z-Score'] = np.nanmax(biased_z_score_array)
                    curr_df['Biased Z-Score at 10%'] = biased_z_score_array[z_score_biased['Num_Residues'] >= float(len(contact_scorer_any.query_structure.seq[contact_scorer_any.best_chain])) * 0.10][0]
                    curr_df['Biased Z-Score at 30%'] = biased_z_score_array[z_score_biased['Num_Residues'] >= float(len(contact_scorer_any.query_structure.seq[contact_scorer_any.best_chain])) * 0.30][0]
                    curr_df['AUC Biased Z-Score'] = biased_scw_z_auc
                    plot_z_scores(z_score_biased, z_score_plot_fn.format('Biased'))
                    z_score_unbiased, unbiased_w2_ave, unbiased_scw_z_auc = contact_scorer_any.score_clustering_of_contact_predictions(
                        bias=False, file_path=z_score_fn.format('Unbiased'), w2_ave_sub=unbiased_w2_ave, processes=10)
                    unbiased_z_score_array = np.array(pd.to_numeric(z_score_unbiased['Z-Score'], errors='coerce'))
                    curr_df['Max Unbiased Z-Score'] = np.nanmax(unbiased_z_score_array)
                    curr_df['Unbiased Z-Score at 10%'] = unbiased_z_score_array[z_score_unbiased['Num_Residues'] >= float(len(contact_scorer_any.query_structure.seq[contact_scorer_any.best_chain])) * 0.10][0]
                    curr_df['Unbiased Z-Score at 30%'] = unbiased_z_score_array[z_score_unbiased['Num_Residues'] >= float(len(contact_scorer_any.query_structure.seq[contact_scorer_any.best_chain])) * 0.30][0]
                    curr_df['AUC Unbiased Z-Score'] = unbiased_scw_z_auc
                    plot_z_scores(z_score_unbiased, z_score_plot_fn.format('Unbiased'))
                    # Record execution times
                    curr_df['Total Time'] = total_time
                    # Record static data for this protein
                    curr_df['Protein'] = p_id
                    curr_df['Method'] = method
                    curr_df['Alignment Size'] = summary['Filtered_Alignment'].values[summary['Protein_ID'] == p_id][0]
                    curr_df.to_csv(curr_fn, sep='\t', header=True, index=False, columns=output_columns)
                    temp_data = os.path.join(method_dir, 'unique_node_data')
                    if os.path.isdir(temp_data):
                        for temp_fn in os.listdir(temp_data):
                            if not (temp_fn.endswith("_pair_rank_filtered_average_product_corrected_mutual_information_score.npz") or
                                    temp_fn.endswith("_pair_rank_match_mismatch_entropy_angle_score.npz")):
                                os.remove(os.path.join(temp_data, temp_fn))
                    print('Metrics meastured for {} covariance for: {}'.format(method, p_id))
                # Set the dataframe for this protein, or update it with additional evaluation data
                if protein_df is None:
                    protein_df = curr_df
                else:
                    protein_df = protein_df.append(curr_df)
            # Write the evaluation data for this protein to file
            protein_df.to_csv(protein_df_fn, sep='\t', header=True, index=False, columns=output_columns)
        # Set the dataframe for this protein, or update it with additional evaluation data
        if small_comparison_df is None:
            small_comparison_df = protein_df
        else:
            small_comparison_df = small_comparison_df.append(protein_df)
    # Write out final comparison data so it can be loaded later for generating figures.
    small_comparison_df['Protein Length'] = small_comparison_df['Protein'].apply(lambda x: generator.protein_data[x]['Length'])
    small_comparison_df.to_csv(small_comparison_fn, sep='\t', header=True, index=False, columns=output_columns)

Removing gaps took 0.08315808375676473 min


/home/daniel/anaconda3/envs/PyET3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11066.
  PDBConstructionWarning)
/home/daniel/anaconda3/envs/PyET3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 11103.
  PDBConstructionWarning)
/home/daniel/anaconda3/envs/PyET3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11145.
  PDBConstructionWarning)


Importing the PDB file took 0.004109783967336019 min
Removing gaps took 0.08719699382781983 min


/home/daniel/anaconda3/envs/PyET3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11066.
  PDBConstructionWarning)
/home/daniel/anaconda3/envs/PyET3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 11103.
  PDBConstructionWarning)
/home/daniel/anaconda3/envs/PyET3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:91: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11145.
  PDBConstructionWarning)


Importing the PDB file took 0.003514440854390462 min
Mapping query sequence and pdb took 0.09292073249816894 min
Constructing internal representation took 0.014908262093861898 min


...

100%|██████████| 318003/318003 [04:34<00:00, 1159.66distances/s]


Constructing tree took: 0.1188251535097758 min


  0%|          | 0/1595 [00:00<?, ?characterizations/s]

It took 69.16111445426941 seconds to identify matches and mismatches.


 89%|████████▊ | 1414/1595 [5:24:21<8:31:35, 169.59s/characterizations]  

# Comparing Execution Time for ET-MIp and cET-MIp
The time to compute the trace for the full phylogenetic tree and the trace constrained to a subset of the top levels should take significantly less time to compute, here we evaluate if that is in fact the case or not.

## Data Cleaning
At least one protein in this data set has a very small alignment and could not be evaluated by cET-MIp because the tree was too small to each the levels set for other proteins. Here we remove those proteins.

In addition since this analysis focuses on times and there are many other types of data (some of which cause redundancies in the time data), we will use this opportunity to subset the data and drop duplicates.

Finally, for some it will be more informative to view execution time in terms of minutes or hours, as opposed to the originally reported seconds, so we will add columns for these units as well.

In [14]:
protein_method_groups = small_comparison_df[['Protein', 'Method']].drop_duplicates().groupby('Protein').count()
method_max = protein_method_groups['Method'].max()
proteins_to_keep = protein_method_groups.index[protein_method_groups['Method'] == method_max]
comparable_method_proteins = small_comparison_df[small_comparison_df['Protein'].isin(proteins_to_keep)]
time_columns = ['Protein', 'Protein Length', 'Alignment Size', 'Method', 'Init Time', 'Import Time', 'Dist Tree Time',
                'Trace Time', 'Total Time']
time_subset_df = comparable_method_proteins.loc[comparable_method_proteins['Method'].isin(['ET-MIp', 'cET-MIp']), time_columns]
time_subset_df['Total Time (min)'] = time_subset_df['Total Time'].apply(lambda x: x / 60.0)
time_subset_df['Total Time (hr)'] = time_subset_df['Total Time (min)'].apply(lambda x: x / 60.0)
time_columns += ['Total Time (min)', 'Total Time (hr)']
time_subset_df = time_subset_df.drop_duplicates(subset=None, inplace=False, keep='first')
time_subset_df.to_csv(os.path.join(small_set_out_dir, 'Small_Time_Comaprison_Data.csv'), sep='\t', header=True, index=False,
                      columns=time_columns)

## Time Comparison
Now that only comparable proteins are present in the data we compare the runtime of individual proteins by method, ordered by their length and the size of their alignemnts.

In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
# Displayed by protein length order
protein_length_order = comparable_method_proteins.sort_values('Protein Length')['Protein'].unique()
protein_length_time_plot = sns.barplot(x='Protein', y='Total Time', hue='Method', order=protein_length_order,
                                       hue_order=['ET-MIp', 'cET-MIp'], data=time_subset_df)
protein_length_time_plot.set_xticklabels(protein_length_time_plot.get_xticklabels(), rotation=90)
protein_length_time_plot.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.savefig(os.path.join(small_set_out_dir, 'Protein_Length_Time_Comparison_Sec.png'), bbox_inches='tight', transparent=True, dpi=300)
plt.close()
protein_length_time_plot = sns.barplot(x='Protein', y='Total Time (min)', hue='Method', order=protein_length_order,
                                       hue_order=['ET-MIp', 'cET-MIp'], data=time_subset_df)
protein_length_time_plot.set_xticklabels(protein_length_time_plot.get_xticklabels(), rotation=90)
protein_length_time_plot.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.savefig(os.path.join(small_set_out_dir, 'Protein_Length_Time_Comparison_Min.png'), bbox_inches='tight', transparent=True, dpi=300)
plt.close()
protein_length_time_plot = sns.barplot(x='Protein', y='Total Time (hr)', hue='Method', order=protein_length_order,
                                       hue_order=['ET-MIp', 'cET-MIp'], data=time_subset_df)
protein_length_time_plot.set_xticklabels(protein_length_time_plot.get_xticklabels(), rotation=90)
protein_length_time_plot.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.savefig(os.path.join(small_set_out_dir, 'Protein_Length_Time_Comparison_Hr.png'), bbox_inches='tight', transparent=True, dpi=300)
plt.close()
# Displayed by alignment size order
protein_alignment_order = comparable_method_proteins.sort_values('Alignment Size')['Protein'].unique()
protein_alignment_time_plot = sns.barplot(x='Protein', y='Total Time', hue='Method', order=protein_alignment_order,
                                          hue_order=['ET-MIp', 'cET-MIp'], data=time_subset_df)
protein_alignment_time_plot.set_xticklabels(protein_alignment_time_plot.get_xticklabels(), rotation=90)
protein_alignment_time_plot.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.savefig(os.path.join(small_set_out_dir, 'Protein_Alignment_Time_Comparison_Sec.png'), bbox_inches='tight', transparent=True, dpi=300)
plt.close()
protein_alignment_time_plot = sns.barplot(x='Protein', y='Total Time (min)', hue='Method', order=protein_alignment_order,
                                          hue_order=['ET-MIp', 'cET-MIp'], data=time_subset_df)
protein_alignment_time_plot.set_xticklabels(protein_alignment_time_plot.get_xticklabels(), rotation=90)
protein_alignment_time_plot.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.savefig(os.path.join(small_set_out_dir, 'Protein_Alignment_Time_Comparison_Min.png'), bbox_inches='tight', transparent=True, dpi=300)
plt.close()
protein_alignment_time_plot = sns.barplot(x='Protein', y='Total Time (hr)', hue='Method', order=protein_alignment_order,
                                          hue_order=['ET-MIp', 'cET-MIp'], data=time_subset_df)
protein_alignment_time_plot.set_xticklabels(protein_alignment_time_plot.get_xticklabels(), rotation=90)
protein_alignment_time_plot.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.savefig(os.path.join(small_set_out_dir, 'Protein_Alignment_Time_Comparison_Hr.png'), bbox_inches='tight', transparent=True, dpi=300)
plt.close()

Overall comparison of time by method.

In [16]:
# Comparison of ET-MIp and cET-MIp total computation time (sec).
protein_method_comp_plot = sns.boxplot(x='Method', y='Total Time', order=['ET-MIp', 'cET-MIp'], data=time_subset_df)
protein_method_comp_plot.set_xticklabels(protein_method_comp_plot.get_xticklabels(), rotation=90)
plt.savefig(os.path.join(small_set_out_dir, 'Protein_Method_Time_Comparison_Sec.png'), bbox_inches='tight', transparent=True, dpi=300)
plt.close()
# Comparison of ET-MIp and cET-MIp total computation time (sec).
protein_method_comp_plot = sns.boxplot(x='Method', y='Total Time (min)', order=['ET-MIp', 'cET-MIp'], data=time_subset_df)
protein_method_comp_plot.set_xticklabels(protein_method_comp_plot.get_xticklabels(), rotation=90)
plt.savefig(os.path.join(small_set_out_dir, 'Protein_Method_Time_Comparison_Min.png'), bbox_inches='tight', transparent=True, dpi=300)
plt.close()
# Comparison of ET-MIp and cET-MIp total computation time (sec).
protein_method_comp_plot = sns.boxplot(x='Method', y='Total Time (hr)', order=['ET-MIp', 'cET-MIp'], data=time_subset_df)
protein_method_comp_plot.set_xticklabels(protein_method_comp_plot.get_xticklabels(), rotation=90)
plt.savefig(os.path.join(small_set_out_dir, 'Protein_Method_Time_Comparison_Hr.png'), bbox_inches='tight', transparent=True, dpi=300)
plt.close()
# Statistical comparison
from scipy.stats import wilcoxon
et_mip_sub_df = time_subset_df[time_subset_df['Method'] == 'ET-MIp']
cet_mip_sub_df = time_subset_df[time_subset_df['Method'] == 'cET-MIp']
sec_stat, sec_p_val = wilcoxon(x=et_mip_sub_df['Total Time'], y=cet_mip_sub_df['Total Time'], zero_method='wilcox')
min_stat, min_p_val = wilcoxon(x=et_mip_sub_df['Total Time (min)'], y=cet_mip_sub_df['Total Time (min)'], zero_method='wilcox')
hr_stat, hr_p_val = wilcoxon(x=et_mip_sub_df['Total Time (hr)'], y=cet_mip_sub_df['Total Time (hr)'], zero_method='wilcox')
time_statistics = {'Time Unit': ['sec', 'min', 'hr'], 'Statistic': [sec_stat, min_stat, hr_stat], 'P-Value': [sec_p_val, min_p_val, hr_p_val]}
pd.DataFrame(time_statistics).to_csv(os.path.join(small_set_out_dir, 'Small_Time_Comaprison_Statistics.csv'), sep='\t', header=True,
                                     index=False, columns=['Time Unit', 'Statistic', 'P-Value'])

## Method Comparison
We now begin comparing methods based on their ability to predict the structural contacts in the proteins in this test set. There is an important consideration in the case of sequence separation and different measures by which to compare the methods.

### Data Cleaning
These data need an additional cleaning step beyond what was performed for the timing comparison. Since there are multiple categories of sequence separation and some proteins may not have any True Positive contacts for a category the scoring for that protein is incomplete. We will remove all such proteins from the comparison, performing the clean up separate for each metric of success. Another contributing factor which necessitates this kind of cleaning is assessment of the top K predictions for a protein or best L/K predictions which for poor predictions may not include any predictions of True Positives.

### Sequence Separation
One important consideration for the difficulty of prediction and interest in predictions is the distance between the residues for which coupling was predicted. As has been documented in the literature, especially in the CASP competitions, there are several categories of prediction:
* Neighbors (1 - 5 residues apart) - This is the least interesting category of predictions. It is highly likely that residues this close together will show covariance signal. Predicting two residues are in contact that are this close together is trivial and uninformative.
* Short (6 - 12 residues apart) - This is also not a very interesting type of prediction. Residues this close in proximity can be more easily modeled by alogrithms which focus on 2D protein structure modeling (identifying beta sheets, alpha helices, etc.).
* Medium (13 - 24 residues apart) - This is a more interesting type of prediction. The resiudes in this range of separation are on the edge of the 2D protein structure prediction range.
* Long (24 and more residues apart) - The most interesting category of predictions. Resiudes this far apart are not easily modeled by 2D protein structure modeling systems. They are also very useful for 3D and 4D protein structure prediction becausae they provide constraints on potential protein (similar to NMR data) folds which makes protein folding a more tractable problem for modelers.
* Any/All - All categories can be considered at once, this provides a summary value, but is often skewed by one particularly good category of predictions.

### Metrics of Success
* AUROC - This measures the True Positive Rate vs the False Positive Rate of prediction, it can be considered a measure of the accuracy of the measure. This can be strongly influenced by the class imbalance which is present when predicting structural contacts since there are many fewer contacts than non-contacts. The True Positive case is if the C-beta of two amino acids is within 8.0 Angstroms of one another (as is done in the CASP competitions).

In [19]:
auroc_columns = ['Protein', 'Method', 'Sequence_Separation', 'AUROC']
protein_auroc_groups = comparable_method_proteins[auroc_columns].drop_duplicates().groupby('Protein')['AUROC'].apply(
    lambda x: not x.isnull().any())
complete_proteins = protein_auroc_groups.index[protein_auroc_groups.values]
comparable_auroc_proteins = small_comparison_df[small_comparison_df['Protein'].isin(complete_proteins)]
auroc_protein_length_order = [x for x in protein_length_order if x in complete_proteins]
auroc_protein_alignment_order = [x for x in protein_alignment_order if x in complete_proteins]
auroc_subset_df = comparable_auroc_proteins.loc[:, auroc_columns].drop_duplicates()
# Plot the methods vs AUROC per protein ordered by protein length
auroc_subset_df.to_csv(os.path.join(small_set_out_dir, 'Small_AUROC_Comaprison_Data.csv'), sep='\t', header=True, index=False,
                       columns=auroc_columns)
protein_order_auroc_plot = sns.catplot(x="Protein", y="AUROC", hue="Method", row="Sequence_Separation", data=auroc_subset_df, kind="bar",
                                       ci=None, order=auroc_protein_length_order, hue_order=method_order, legend=True, legend_out=True)
protein_order_auroc_plot.set_xticklabels(auroc_protein_length_order, rotation=90)
protein_order_auroc_plot.savefig(os.path.join(small_set_out_dir, 'Protein_Method_AUROC_Comparison_Protein_Length_Order.png'),
                                 bbox_inches='tight', transparent=True, dpi=300)
plt.close()
# Plot the methods vs AUROC per protein ordered by protein alignment size
alignment_order_auroc_plot = sns.catplot(x="Protein", y="AUROC", hue="Method", row="Sequence_Separation", data=auroc_subset_df, kind="bar",
                                       ci=None, order=auroc_protein_alignment_order, hue_order=method_order, legend=True, legend_out=True)
alignment_order_auroc_plot.set_xticklabels(auroc_protein_alignment_order, rotation=90)
alignment_order_auroc_plot.savefig(os.path.join(small_set_out_dir, 'Protein_Method_AUROC_Comparison_Alignment_Size_Order.png'),
                                   bbox_inches='tight', transparent=True, dpi=300)
plt.close()
# Plot the methods vs AUROC grouped together to see overall trends
overall_auroc_plot = sns.boxplot(x="Sequence_Separation", y="AUROC", hue="Method", data=auroc_subset_df,
                                 order=sequence_separation_order, hue_order=method_order)
overall_auroc_plot.set_xticklabels(sequence_separation_order, rotation=90)
overall_auroc_plot.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.savefig(os.path.join(small_set_out_dir, 'Protein_Method_AUROC_Comparison.png'), bbox_inches='tight', transparent=True, dpi=300)
plt.close()
# Compute statistics comparing methods at each sequence separation
auroc_statistics = {'Sequence Separation': [], 'Method 1': [], 'Method 2': [], 'Statistic': [], 'P-Value': []}
for sep in sequence_separation_order:
    sep_auroc_subset_df = auroc_subset_df.loc[auroc_subset_df['Sequence_Separation'] == sep, :]
    for i in range(len(method_order)):
        m1_sep_auroc_subset_df = sep_auroc_subset_df.loc[sep_auroc_subset_df['Method'] == method_order[i], :]
        for j in range(i + 1, len(method_order)):
            m2_sep_auroc_subset_df = sep_auroc_subset_df.loc[sep_auroc_subset_df['Method'] == method_order[j], :]
            stat, p_val = wilcoxon(x=m1_sep_auroc_subset_df['AUROC'], y=m2_sep_auroc_subset_df['AUROC'], zero_method='wilcox')
            auroc_statistics['Sequence Separation'].append(sep)
            auroc_statistics['Method 1'].append(method_order[i])
            auroc_statistics['Method 2'].append(method_order[j])
            auroc_statistics['Statistic'].append(stat)
            auroc_statistics['P-Value'].append(p_val)
pd.DataFrame(auroc_statistics).to_csv(os.path.join(small_set_out_dir, 'Small_AUROC_Comaprison_Statistics.csv'), sep='\t', header=True,
                                      index=False, columns=['Sequence Separation', 'Method 1', 'Method 2', 'Statistic', 'P-Value'])

### Metrics of Success (Continued)
* AUPRC - This measures the Precision vs the Recall of the predictions, it can be considered a measure of the accuracy of the measure. This is less strongly influenced by the class imbalance which is present when predicting structural contacts since there are many fewer contacts than non-contacts. The True Positive case is if the C-beta of two amino acids is within 8.0 Angstroms of one another (as is done in the CASP competitions).

In [20]:
auprc_columns = ['Protein', 'Method', 'Sequence_Separation', 'AUPRC']
protein_auprc_groups = comparable_method_proteins[auprc_columns].drop_duplicates().groupby('Protein')['AUPRC'].apply(
    lambda x: not x.isnull().any())
complete_proteins = protein_auprc_groups.index[protein_auprc_groups.values]
comparable_auprc_proteins = small_comparison_df[small_comparison_df['Protein'].isin(complete_proteins)]
auprc_protein_length_order = [x for x in protein_length_order if x in complete_proteins]
auprc_protein_alignment_order = [x for x in protein_alignment_order if x in complete_proteins]
auprc_subset_df = comparable_auprc_proteins.loc[:, auprc_columns].drop_duplicates()
# Plot the methods vs AUPRC per protein ordered by protein length
auprc_subset_df.to_csv(os.path.join(small_set_out_dir, 'Small_AUPRC_Comaprison_Data.csv'), sep='\t', header=True, index=False,
                       columns=auprc_columns)
protein_order_auprc_plot = sns.catplot(x="Protein", y="AUPRC", hue="Method", row="Sequence_Separation", data=auprc_subset_df, kind="bar",
                                       ci=None, order=auprc_protein_length_order, hue_order=method_order, legend=True, legend_out=True)
protein_order_auprc_plot.set_xticklabels(auprc_protein_length_order, rotation=90)
protein_order_auprc_plot.savefig(os.path.join(small_set_out_dir, 'Protein_Method_AUPRC_Comparison_Protein_Length_Order.png'),
                                 bbox_inches='tight', transparent=True, dpi=300)
plt.close()
# Plot the methods vs AUPRC per protein ordered by protein alignment size
alignment_order_auprc_plot = sns.catplot(x="Protein", y="AUPRC", hue="Method", row="Sequence_Separation", data=auprc_subset_df, kind="bar",
                                       ci=None, order=auprc_protein_alignment_order, hue_order=method_order, legend=True, legend_out=True)
alignment_order_auprc_plot.set_xticklabels(auprc_protein_alignment_order, rotation=90)
alignment_order_auprc_plot.savefig(os.path.join(small_set_out_dir, 'Protein_Method_AUPRC_Comparison_Alignment_Size_Order.png'),
                                   bbox_inches='tight', transparent=True, dpi=300)
plt.close()
# Plot the methods vs AUPRC grouped together to see overall trends
overall_auprc_plot = sns.boxplot(x="Sequence_Separation", y="AUPRC", hue="Method", data=auprc_subset_df,
                                 order=sequence_separation_order, hue_order=method_order)
overall_auprc_plot.set_xticklabels(sequence_separation_order, rotation=90)
overall_auprc_plot.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)
plt.savefig(os.path.join(small_set_out_dir, 'Protein_Method_AUPRC_Comparison.png'), bbox_inches='tight', transparent=True, dpi=300)
plt.close()
# Compute statistics comparing methods at each sequence separation
auprc_statistics = {'Sequence Separation': [], 'Method 1': [], 'Method 2': [], 'Statistic': [], 'P-Value': []}
for sep in sequence_separation_order:
    sep_auprc_subset_df = auprc_subset_df.loc[auprc_subset_df['Sequence_Separation'] == sep, :]
    for i in range(len(method_order)):
        m1_sep_auprc_subset_df = sep_auprc_subset_df.loc[sep_auprc_subset_df['Method'] == method_order[i], :]
        for j in range(i + 1, len(method_order)):
            m2_sep_auprc_subset_df = sep_auprc_subset_df.loc[sep_auprc_subset_df['Method'] == method_order[j], :]
            stat, p_val = wilcoxon(x=m1_sep_auprc_subset_df['AUPRC'], y=m2_sep_auprc_subset_df['AUPRC'], zero_method='wilcox')
            auprc_statistics['Sequence Separation'].append(sep)
            auprc_statistics['Method 1'].append(method_order[i])
            auprc_statistics['Method 2'].append(method_order[j])
            auprc_statistics['Statistic'].append(stat)
            auprc_statistics['P-Value'].append(p_val)
pd.DataFrame(auprc_statistics).to_csv(os.path.join(small_set_out_dir, 'Small_AUPRC_Comaprison_Statistics.csv'), sep='\t', header=True,
                                      index=False, columns=['Sequence Separation', 'Method 1', 'Method 2', 'Statistic', 'P-Value'])

* Precision at K - 

* Recall at K - 

* F1 at K - 

* Structural Cluster Weighting Z-Score - 